# Importing libraries

In [ ]:
import pandas as pd
import os
import glob
import re
import string
from google.colab import drive
import nltk
import numpy as np
import scipy.sparse
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from tensorflow import keras
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Extracting data from each folders which contain .txt files

In [34]:
#provide link based on the location of the data set
path = '/content/drive/MyDrive/machinelearning data sets/datasetsCW/bbc'#location of where the data set is stored.
categories_list = os.listdir('/content/drive/MyDrive/machinelearning data sets/datasetsCW/bbc') #['politics', 'entertainment', 'sport', 'tech', 'business']
articles = [] #for storing all the articles
categories = [] #for storing all the categories (eg:-business, politics, tech, etc.)
for category in categories_list:# one each iterations the data inside categories_list will send into category variable.
  paths =  glob.glob(os.path.join(path , category , '*.txt'), recursive=True)# (paths=path/category/ANY.txt )files if any txt file then store the content into paths variable
  for idx_file in range(len(paths)):
    categories.append(category)#append the folder name in the categories list
    with open(paths[idx_file], mode = 'r', encoding = "ISO-8859-1") as file:
      articles.append(file.read())#open the file, read the content inside it and append the content to articles varirable
le=len(articles)
print('total',le,'file in articles folders')

total 2225 file in articles folders


# Creating data frame of the data which we recieved after extraction.

In [35]:
df=pd.DataFrame({'Categories':categories,'Articles':articles})# we can now convert the data categories & articles list into a dataframe.
Y=df['Categories']#extracting the Categories column into a variable for later use.
df

,Categories,Articles
0,politics,Civil servants in strike ballot\n\nThe UK's bi...
1,politics,Brown names 16 March for Budget\n\nChancellor ...
2,politics,Defection timed to hit tax pledge\n\nWith impe...
3,politics,Jowell rejects 'Las Vegas' jibe\n\nThe Secreta...
4,politics,Blair backs 'pre-election budget'\n\nTony Blai...
...,...,...
2220,business,Venezuela reviews foreign deals\n\nVenezuela i...
2221,business,China now top trader with Japan\n\nChina overt...
2222,business,Tsunami cost hits Jakarta shares\n\nThe stock ...
2223,business,Cairn shares up on new oil find\n\nShares in C...


In [5]:
df.info()#checking the null-values = no null values
Y=pd.DataFrame(Y)
Y

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categories  2225 non-null   object
 1   Articles    2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


,Categories
0,politics
1,politics
2,politics
3,politics
4,politics
...,...
2220,business
2221,business
2222,business
2223,business


# Data preprocessing(removing all the unnecessary characters from the data)

In [6]:

def cleanText(text):
  stemmer = WordNetLemmatizer()#creating an object of WordNetLemmatizer
  text=text.lower()#convertin the text into lowercase
  text=re.sub('\[.*?\]','',text)#remove text in square brackets
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)#remove punctuation markes
  text=re.sub('\w*\d\w*','',text) #remove digits or digits which is surrounded by texts
  text=re.sub('\[''""...]','',text)#remove quotation marks
  text=re.sub('\n','',text)#remove \n if any
  text= text.split()#split words
  text = [stemmer.lemmatize(word) for word in text]#from each text a single word is taken and on that word lemmatiztion is performed and stored back to text
  text = ' '.join(text)# adds space to the words
  return text

In [7]:
cleandata=pd.DataFrame(df.Articles.apply(lambda x: cleanText(x)))#CleanText function is applied to every single row of Article column inside the data frame
cleandata['Categories']=Y['Categories']#merging the article and Categories columns to gather.
cleandata

,Articles,Categories
0,civil servant in strike ballotthe uk biggest c...,politics
1,brown name march for budgetchancellor gordon b...,politics
2,defection timed to hit tax pledgewith impeccab...,politics
3,jowell reject la vega jibethe secretary of sta...,politics
4,blair back preelection budgettony blair ha bac...,politics
...,...,...
2220,venezuela review foreign dealsvenezuela is to ...,business
2221,china now top trader with japanchina overtook ...,business
2222,tsunami cost hit jakarta sharesthe stock marke...,business
2223,cairn share up on new oil findshares in cairn ...,business


In [8]:
cleandata.Articles

0       civil servant in strike ballotthe uk biggest c...
1       brown name march for budgetchancellor gordon b...
2       defection timed to hit tax pledgewith impeccab...
3       jowell reject la vega jibethe secretary of sta...
4       blair back preelection budgettony blair ha bac...
                              ...                        
2220    venezuela review foreign dealsvenezuela is to ...
2221    china now top trader with japanchina overtook ...
2222    tsunami cost hit jakarta sharesthe stock marke...
2223    cairn share up on new oil findshares in cairn ...
2224    unilever shake up a profit slipsanglodutch con...
Name: Articles, Length: 2225, dtype: object

# 1.Feature engineering - Count vectorizer
Now, converting the data into document term metix through count vectorizer

In [9]:
from pandas.core.frame import DataFrame
cv=CountVectorizer(stop_words='english')
Xcv=cv.fit_transform(cleandata.Articles).toarray()
Xcv=pd.DataFrame(Xcv,columns=cv.get_feature_names_out())
Xcv.head()

,aa,aaa,aac,aaca,aaced,aack,aadc,aaliyah,aaliyahs,aaltra,...,zoomsmore,zooropa,zornotza,zorro,zubair,zuluaga,zurich,zutons,zvonareva,zvyagintsev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 2.Feature engineering - TFIDF 
Now, using second feature Tfidf

In [38]:
tfidfv=TfidfVectorizer(stop_words='english')
Xtfidf=tfidfv.fit_transform(cleandata.Articles)
Xtfidf

<2225x35463 sparse matrix of type '<class 'numpy.float64'>'
	with 305813 stored elements in Compressed Sparse Row format>

#3. Feature engineering - Ngram

In [11]:
cvngram=TfidfVectorizer(min_df=7,ngram_range=(2,2))
X=cvngram.fit_transform(cleandata.Articles)
X

<2225x13805 sparse matrix of type '<class 'numpy.float64'>'
	with 290547 stored elements in Compressed Sparse Row format>

#Combining all the three features

In [12]:
features=FeatureUnion([('tfidfv',tfidfv),('cv',cv),('cvngram',cvngram)])
X=features.fit_transform(cleandata.Articles)
X1=SelectKBest(score_func=f_classif,k= 3000).fit_transform(X,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Kfold 

In [13]:
kf=KFold(n_splits=5,random_state=42, shuffle=True)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

#Model selection Dictionary

In [14]:
modelselection={
  'SupportVectorMachine': {
  'model': svm.SVC(gamma='auto',random_state=42),
  'params':{
    'C':[1,5,10,20],
    'kernel':['rbf','linear']
    }
  },
   'RandomForsetClassifier': {
  'model': RandomForestClassifier(random_state=42),
  'params':{
    'n_estimators':[20,50,60]
    }
  },
  'LogesticRegression': {
  'model': LogisticRegression(solver='liblinear',multi_class='auto',random_state=42),
  'params':{
    'C':[5,10,20]
    }
  },
  'NaiveBayes-Multimonial':{
  'model':MultinomialNB(),
  'params':{
    }
  },
  'aiveBayes-Bernoulli': {
  'model': BernoulliNB(),
  'params':{

    }
  },
 'VotingClassifier': {
  'model': VotingClassifier(
      estimators=[('lr',LogisticRegression(solver='liblinear',multi_class='auto',random_state=42)),('rfc',RandomForestClassifier(random_state=42)),('svm',svm.SVC(gamma='auto',random_state=42)),('nbm',MultinomialNB()),('nbb',BernoulliNB())],
      voting='hard'
  ),
  'params':{

    }
  },
  
}


In [15]:
score=[]
for train, test in kf.split(X1):
  Xtrain,Xtest,Ytrain,Ytest=X1[train],X1[test],cleandata.Categories[train], cleandata.Categories[test]
  for modelname, mp in modelselection.items():
    clf=GridSearchCV(mp['model'],mp['params'],return_train_score=False)
    clf.fit(Xtrain,Ytrain)
    score.append({
      'model':modelname,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_
    })
  print('number of iterations')
  print('--------------------')

number of iterations
--------------------
number of iterations
--------------------
number of iterations
--------------------
number of iterations
--------------------
number of iterations
--------------------


In [16]:
pd.DataFrame(score)

,model,best_score,best_params
0,SupportVectorMachine,0.966854,"{'C': 10, 'kernel': 'rbf'}"
1,RandomForsetClassifier,0.953933,{'n_estimators': 60}
2,LogesticRegression,0.969663,{'C': 5}
3,NaiveBayes-Multimonial,0.970225,{}
4,aiveBayes-Bernoulli,0.967978,{}
5,VotingClassifier,0.974719,{}
6,SupportVectorMachine,0.964045,"{'C': 5, 'kernel': 'rbf'}"
7,RandomForsetClassifier,0.950000,{'n_estimators': 60}
8,LogesticRegression,0.964045,{'C': 10}
9,NaiveBayes-Multimonial,0.971348,{}


# CrossValidation

In [32]:
values=[]
accuracy=[]
classi=[]
recall=[]
precision=[]
f1=[]
vc=VotingClassifier(estimators=[('lr',LogisticRegression(solver='liblinear',multi_class='auto',random_state=42)),('rfc',RandomForestClassifier(random_state=42)),('svm',svm.SVC(gamma='auto',random_state=42)),('nbm',MultinomialNB()),('nbb',BernoulliNB())],
     voting='hard')
totalscore=0
for train, test in kf.split(X1):
  Xtrain,Xtest,Ytrain,Ytest=X1[train],X1[test],cleandata.Categories[train], cleandata.Categories[test]
  vc.fit(Xtrain,Ytrain)
  predictedvalues=vc.predict(Xtest)
  values.append(predictedvalues)
  score=accuracy_score(Ytest,predictedvalues)
  recallvar=recall_score(Ytest,predictedvalues,average='macro')
  recall.append(recallvar)
  prevar=precision_score(Ytest,predictedvalues,average='macro')
  precision.append(prevar)
  accuracy.append(score)
  f1.append(f1_score(Ytest,predictedvalues,average='macro'))
print('The accuracy is ',np.mean(accuracy))
print('The macro average precision is ',np.mean(precision))
print('The macro average recall is ',np.mean(recall))
print('The macro average f1-score is ',np.mean(f1))

The accuracy is  0.9770786516853931
The macro average precision is  0.9778004664077564
The macro average recall is  0.9761368400689152
The macro average f1-score is  0.9768409407009522
